<a href="https://colab.research.google.com/github/MohamadRayi/MachineLearningTasks/blob/main/Task%20Lecture%2013%20Machine%20Learning/BAB%203/Fine_tuning_a_model_with_the_Trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mohamad Rayi Dwi Putra

1103213070

Task Lecture 13

Fine_tuning_a_model_with_the_Trainer_API

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [4]:
# Mengimpor pustaka PyTorch
import torch
# Mengimpor kelas optimizer AdamW dan kelas-kelas terkait dari pustaka Transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Menentukan model yang akan digunakan
checkpoint = "bert-base-uncased"
# Menginisialisasi tokenizer berdasarkan model yang telah ditentukan sebelumnya
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Menginisialisasi model untuk klasifikasi urutan teks berdasarkan model yang telah ditentukan sebelumnya
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Menyiapkan urutan teks yang akan diproses
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
# Membuat batch dari urutan teks dengan padding dan truncation
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# Menambahkan label-label yang sesuai ke dalam batch
batch["labels"] = torch.tensor([1, 1])

# Menginisialisasi optimizer AdamW dengan parameter-parameter model
optimizer = AdamW(model.parameters())
# Menghitung loss dan melakukan backpropagation
loss = model(**batch).loss
loss.backward()
# Melakukan satu langkah optimisasi (update parameter)
optimizer.step()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
# Mengimpor fungsi yang diperlukan dari pustaka datasets
from datasets import load_dataset

# Mengimpor kelas-kelas yang diperlukan dari pustaka transformers
from transformers import AutoTokenizer, DataCollatorWithPadding

# Memuat dataset GLUE dengan metode "mrpc"
# Dataset GLUE (General Language Understanding Evaluation) terdiri dari beberapa tugas NLP, termasuk MRPC (Microsoft Research Paraphrase Corpus)
raw_datasets = load_dataset("glue", "mrpc")

# Memilih model BERT yang akan digunakan
# Model yang dipilih adalah 'bert-base-uncased', yaitu model BERT dengan teks huruf kecil semua
checkpoint = "bert-base-uncased"

# Membuat tokenizer otomatis dari model yang dipilih
# Tokenizer akan digunakan untuk mengubah teks menjadi token yang dapat diproses oleh model BERT
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Mendefinisikan fungsi untuk tokenisasi setiap contoh dalam dataset
# Fungsi ini menerima dua kalimat (sentence1 dan sentence2) dan mengembalikan token yang dipotong sesuai kebutuhan (truncation=True)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Melakukan tokenisasi pada dataset menggunakan fungsi yang telah didefinisikan
# Fungsi map dengan parameter batched=True akan mengaplikasikan fungsi tokenisasi pada setiap batch dalam dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Membuat objek DataCollatorWithPadding untuk menyiapkan batch data dengan padding
# DataCollatorWithPadding memastikan semua urutan dalam batch memiliki panjang yang sama dengan menambahkan padding jika diperlukan
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

In [7]:
# Mendefinisikan fungsi untuk menghitung metrik evaluasi
def compute_metrics(eval_preds):
    # Memuat metrik evaluasi dari pustaka 'evaluate' untuk tugas GLUE dan dataset MRPC
    metric = evaluate.load("glue", "mrpc")

    # Memisahkan logits (output model) dan label dari hasil evaluasi
    logits, labels = eval_preds

    # Menghasilkan prediksi dari logits dengan memilih indeks dengan nilai tertinggi
    predictions = np.argmax(logits, axis=-1)

    # Menghitung metrik menggunakan prediksi dan label
    return metric.compute(predictions=predictions, references=labels)